In [4]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [5]:
periods = list(range(1, 6))
prev_period = dict(zip(periods, [5, 1, 2, 3, 4]))
time_h  = dict(zip(periods, ['00:00~06:00',
                             '06:00~09:00',
                             '09:00~15:00',
                             '15:00~18:00',
                             '18:00~00:00']))
hours   = dict(zip(periods, [6, 3, 6, 3, 6]))
demand  = dict(zip(periods, [15000, 30000, 25000, 40000, 27000])) 

gen_types = [1, 2, 3]
gen_qty = dict(zip(gen_types, [12, 10, 5]))
gen_min = dict(zip(gen_types, [850, 1250, 1500]))
gen_max = dict(zip(gen_types, [2000, 1750, 4000]))
gen_basecost  = dict(zip(gen_types, [1000, 2600, 3000]))
gen_hourcost  = dict(zip(gen_types, [2, 1.3, 3]))
gen_startcost = dict(zip(gen_types, [2000, 1000, 500]))

hyd_types = ['A', 'B']
hyd_oper_lvl = dict(zip(hyd_types, [900, 1400]))
hyd_hourcost = dict(zip(hyd_types, [90, 150]))
hyd_rese_red = dict(zip(hyd_types, [0.31, 0.47]))
hyd_startcost = dict(zip(hyd_types, [1500, 1200]))

In [29]:
model = gp.Model('Hydro Power')

# add vars
n = model.addVars(gen_types, periods,  # qty of generators of type i already on in period t
                  vtype=GRB.INTEGER,
                  name='n')
s = model.addVars(gen_types, periods,  # qty of generators of type i started up in period t 
                  vtype=GRB.INTEGER,
                  name='s')
x = model.addVars(gen_types, periods,  # total output of generators of type i
                  name='x')
h = model.addVars(hyd_types, periods,  # if hydro k is on in period j
                  vtype=GRB.BINARY,
                  name='h')
t = model.addVars(hyd_types, periods,  # if hydro k started up in period j
                  vtype=GRB.BINARY,
                  name='j')
l = model.addVars(periods,             # reservoir level in period j
                  name='l')
p = model.addVars(periods,             # watts used to pump water in period j 
                  name='p')

for i in gen_types:
    for j in periods:
        n[i,j].setAttr('ub', gen_qty[i])        

for j in periods:
    l[j].setAttr('lb', 16)  

# objective function
model.setObjective((gp.quicksum(gen_startcost[i]*s[i,j] 
                                for i in gen_types for j in periods)
                  + gp.quicksum(gen_basecost[i]*hours[j]*n[i,j]
                                for i in gen_types for j in periods)
                  + gp.quicksum(gen_hourcost[i]*hours[j]*(x[i,j] - gen_min[i]*n[i,j])
                                for i in gen_types for j in periods)
                  + gp.quicksum(hyd_hourcost[k]*hours[j]*h[k,j]
                                for k in hyd_types for j in periods)
                  + gp.quicksum(hyd_startcost[k]*t[k,j]
                                for k in hyd_types for j in periods)))

# constraints
model.addConstrs((gp.quicksum(x[i,j] for i in gen_types) 
                + gp.quicksum(hyd_oper_lvl[k]*h[k,j] for k in hyd_types) 
                - p[j] >= demand[j] for j in periods),
                 name='demand')

model.addConstrs((x[i,j] >= gen_min[i]*n[i,j] 
                 for i in gen_types for j in periods),
                name='min_output')

model.addConstrs((x[i,j] <= gen_max[i]*n[i,j] 
                 for i in gen_types for j in periods),
                name='max_output')

model.addConstrs((gp.quicksum(gen_max[i]*n[i,j] for i in gen_types) 
                + gp.quicksum(hyd_oper_lvl[k] for k in hyd_types)
                  >= 1.15*demand[j] for j in periods),
                name='extra_demand')

model.addConstrs((s[i,j] >= n[i,j] - n[i,prev_period[j]] 
                 for i in gen_types for j in periods),
                name='gen_starting')

model.addConstrs((t[k,j] >= h[k,j] - h[k,prev_period[j]] 
                 for k in hyd_types for j in periods),
                name='hyd_starting')

model.addConstrs((l[j] - l[prev_period[j]] - hours[j]*p[j]/3000
                + gp.quicksum(hours[j]*hyd_rese_red[k]*h[k,j] for k in hyd_types)
                  == 0 for j in periods),
                name='rese_level')

model.update()
model.write('Hydro Power.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 70 rows, 75 columns and 205 nonzeros
Model fingerprint: 0x9060a227
Variable types: 25 continuous, 50 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+04, 4e+04]
Found heuristic solution: objective 1164975.0000
Presolve time: 0.00s
Presolved: 70 rows, 75 columns, 205 nonzeros
Variable types: 25 continuous, 50 integer (20 binary)

Root relaxation: objective 9.850143e+05, 28 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 985014.286    0    7 1164975.00 985014.286  15.4%     -    0s
H    0     0                    994935.00000 985014.286  1.00%     -    0s
H    0     0             

In [25]:
print('Total cost: ${:.2f}'.format(model.ObjVal))

for j in periods:
    print('\nPeriod {}'.format(j))
    for i in gen_types:
        if n[i,j].x > 0:
            print('{} Thermal type {} producing {:.0f} MW'.format(n[i,j].x, i, x[i,j].x))
    for k in hyd_types:
        if h[k,j].x > 0:
            print('Hydro type {} producing {:.0f} MW'.format(k, hyd_oper_lvl[k]))

Total cost: $986630.00

Period 1
12.0 Thermal type 1 producing 10565 MW
3.0 Thermal type 2 producing 5250 MW

Period 2
12.0 Thermal type 1 producing 14250 MW
9.0 Thermal type 2 producing 15750 MW

Period 3
12.0 Thermal type 1 producing 10200 MW
9.0 Thermal type 2 producing 15750 MW

Period 4
12.0 Thermal type 1 producing 21350 MW
9.0 Thermal type 2 producing 15750 MW
1.0 Thermal type 3 producing 1500 MW
Hydro type B producing 1400 MW

Period 5
12.0 Thermal type 1 producing 10200 MW
9.0 Thermal type 2 producing 15750 MW
Hydro type B producing 1400 MW


In [26]:
for j in periods:
    print('\nPeriod {}'.format(j))
    for i in gen_types:
        if s[i,j].x>0:
            print('Start {} generators of type {}'.format(s[i,j].x, i))


Period 1

Period 2
Start 6.0 generators of type 2

Period 3

Period 4
Start 1.0 generators of type 3

Period 5


In [27]:
for j in periods:
    if p[j].x > 0:
        print('\nPeriod {}'.format(j))
        print('Pumping using {:.0f}MW'.format(p[j].x))
    


Period 1
Pumping using 815MW

Period 3
Pumping using 950MW

Period 5
Pumping using 350MW


In [28]:
for j in periods:
    if l[j].x > 0:
        print('\nPeriod {}'.format(j))
        print('Reservoir level at {:.2f}m'.format(l[prev_period[j]].x))


Period 1
Reservoir level at 16.00m

Period 2
Reservoir level at 17.63m

Period 3
Reservoir level at 17.63m

Period 4
Reservoir level at 19.53m

Period 5
Reservoir level at 18.12m
